# Data Preparation


Il dataset iniziale è il dataset completo delle interviste effettuate a oltre 400000 utenti.  Visto l'obiettivo di questo studio, ossia una classificazione multiclasse del diabete, verranno escluse dal dataset tutte quelle domande non utili alla generalizzazione, come info personali identificative dell'individuo intervistato o i suoi possedimenti.

Verranno inoltre eliminate tutte quelle colonne derivate.

Il tutto è stato possibile farlo consultando il notebook relativo al dataset.


In [ ]:
import pandas as pd
import numpy as np


df = pd.read_csv('dmml_diabetes_db.csv')
df.info()


Le colonne da eliminare sono state trovate tramite codebook.

In [ ]:
# Colonne derivate da eliminare:
dropped_cols = [
    "_RFHLTH", "_HCVU651", "_RFHYPE5", "_CHOLCHK", "_RFCHOL", "_MICHD",
    "_LTASTH1", "_CASTHM1", "_ASTHMS1", "_DRDXAR1", "_PRACE1", "_MRACE1",
    "_HISPANC", "_RACEG21", "_RACEGR3", "_RACE_G1", "_AGEG5YR", "_AGE65YR",
    "_AGE80", "HTIN4", "HTM4", "WTKG3", "_RFBMI5", "_CHLDCNT", "_EDUCAG",
    "_INCOMG", "_RFSMOK3", "DRNKANY5", "DROCDY3_", "_RFBING5", "_DRNKWEK",
    "_RFDRHV5", "FTJUDA1_", "FRUTDA1_", "BEANDAY_", "GRENDAY_", "ORNGDAY_",
    "VEGEDA1_", "_MISFRTN", "_MISVEGN", "_FRTRESP", "_VEGRESP", "_FRUTSUM",
    "_VEGESUM", "_FRTLT1", "_VEGLT1", "_FRT16", "_VEG23", "_FRUITEX",
    "_VEGETEX", "_TOTINDA", "MAXVO2_", "FC60_", "STRFREQ_", "PAMISS1_",
    "_PAINDX1", "_PA150R2", "_PA300R2", "_PA30021", "_PASTRNG", "_PAREC1",
    "_PASTAE1", "_LMTACT1", "_LMTWRK1", "_LMTSCL1", "_RFSEAT2", "_RFSEAT3",
    "_AIDTST3", "_CHISPNC", "_BMI5"
]
indices = [df.columns.get_loc(col) for col in dropped_cols if col in df.columns]
print(indices)




In [ ]:
df.columns.get_loc('WEIGHT2')

Tenendo BMI tolgo altezza e peso

colonne derivate + colonne non inerenti

In [ ]:

drop_indices = (
    list(range(1, 14))    # 1–13
    + list(range(16, 20))  # 16–19
    + list(range(21, 25))  # 21–24
    + list(range(55, 59))  # 55–58
    + [64] + [65]          # 64, 65
    + [197]                # 197
    + list(range(222, 232))# 222–230
    + [235]                # 235
    + list(range(237, 251))# 237–250
    + list(range(252, 258))# 252–257
    + list(range(259, 263))# 259–262
    + list(range(264, 268))# 264–267
    + list(range(269, 294))# 269–293
    + list(range(296, 298))# 296–297
    + list(range(306, 308))# 306–307
    + list(range(315, 327))# 315–326
    + [329]                # 329
)

# Rimuovi eventuali duplicati e tieni gli indici validi
drop_indices = sorted({i for i in drop_indices if 0 <= i < len(df.columns)})

# Ottengo i nomi delle colonne e elimino
cols_to_drop = [df.columns[i] for i in drop_indices]
df = df.drop(columns=cols_to_drop)

print(f"Eliminate {len(cols_to_drop)} colonne.")  


Accorpamento in una singola colonna per due variabili che riguardano la stessa domanda, effettuata su linea fissa e su linea mobile, che le differenzia

In [ ]:
# Stampa il numero di colonne iniziali
print(f"Colonne iniziali: {df.shape[1]}")

# Definizione nomi colonne
primary_col   = 'NUMADULT'
secondary_col = 'HHADULT'
new_col       = 'NUMADULT_2'

# Controllo esistenza nuova colonna
if new_col in df.columns:
    print(f"La colonna `{new_col}` esiste già. Operazione saltata.")
# Controllo esistenza colonne di origine
elif not {primary_col, secondary_col}.issubset(df.columns):
    missing = [c for c in (primary_col, secondary_col) if c not in df.columns]
    print(f"Impossibile creare `{new_col}`, mancano le colonne: {missing}")
else:
    # Creazione nuova colonna con preferenza per primary_col
    df[new_col] = df[primary_col].combine_first(df[secondary_col])
    # Eliminazione colonne originali in-place
    df.drop(columns=[primary_col, secondary_col], inplace=True)
    print(f"✅ `{new_col}` creata unendo `{primary_col}` e `{secondary_col}`.")
    
# Info post-elaborazione
print(f"Colonne finali: {df.shape[1]}")
print("Verifica presenza colonne originali:")
print(f"- `{primary_col}` presente? {primary_col in df.columns}")
print(f"- `{secondary_col}` presente? {secondary_col in df.columns}")


Butto via le colonne che contengono più del 30% di valori Nan

In [ ]:
# Soglia di NaN (30%)
threshold = 0.30
# Calcola la percentuale di NaN per colonna
nan_ratio = df.isna().mean()
# Trova colonne da eliminare
cols_to_drop = nan_ratio[nan_ratio > threshold].index.tolist()
# Stampa e rimuovi
if cols_to_drop:
    print("Colonne eliminate per eccesso di NaN (> 30%):")
    for col in cols_to_drop:
        print(f"- {col}")
    df = df.drop(columns=cols_to_drop)
else:
    print("Nessuna colonna supera il 30% di valori NaN.")
print(f"Numero colonne attuali: {df.shape[1]}")

In [ ]:
#proviamo a droppare le righe che contengon valori nulli.
print(f"Righe originali: {df.shape[0]}")
df = df.dropna()
print(f"Righe dopo il drop: {df.shape[0]}")

Aggiusto la variabile target

In [ ]:
#droppo le righe che hanno diabete3 pari a 7 o 9 (non lo so/ nessuna risposta)
before = df.shape[0]
df = df[~df['DIABETE3'].isin([7, 9])]
after = df.shape[0]

mapping_DIABETE3 = {
    1: 'Diabetes',
    2: 'NoDiabetes',
    3: 'NoDiabetes',
    4: "PreDiabetes", 
}
df['DIABETE3'] = df['DIABETE3'].map(mapping_DIABETE3)

print(f"Righe eliminate: {before - after}")
print(f"Righe rimanenti: {after}")

Ci occupiamo adesso delle variabili la cui codifica va cambiata per migliorare l'interpretazione nei modelli.

Le variabili relative al consumo di cibo verranno tutte converitite in consumo mensile.

In [ ]:

def freq_food_to_monthly(code):
    # valori missing/refused
    if code in (777, 999) or pd.isna(code):
        return np.nan
    # mai
    if code == 555:
        return 0.0
    # volte per giorno → moltiplico 30 giorni
    if 101 <= code <= 199:
        return (code - 100) * 30.0
    # volte per settimana → moltiplico 4.345 settimane 
    if 201 <= code <= 299:
        return (code - 200) * 4.345
    # meno di 1 al mese → stimo 0.5
    if code == 300:
        return 0.5
    # volte per mese diretto
    if 301 <= code <= 399:
        return code - 300
    # altrimenti missing
    return np.nan

for var in ['FRUITJU1','FRUIT1','FVBEANS','FVGREEN','FVORANG','VEGETAB1']:
    df[f'{var}'] = df[var].apply(freq_food_to_monthly)


In [ ]:
def strength_to_weekly(code):
    if code in (777, 999) or pd.isna(code):
        return np.nan
    if code == 888:
        return 0.0
    if 101 <= code <= 199:
        return code - 100
    if 201 <= code <= 299:
        # converto il valore mensile in frequenza settimanale
        return (code - 200) / 4.345
    return np.nan

df['STRENGTH'] = df['STRENGTH'].apply(strength_to_weekly) 


In [ ]:
def alco_days_to_weekly(code):
    if code in (777, 999) or pd.isna(code):
        return np.nan
    if code == 888:
        return 0.0
    if 101 <= code <= 199:
        # giorni a settimana
        return code - 100
    if 201 <= code <= 299:
        # giorni in 30 giorni → giorni/settimana
        return (code - 200) / 4.345
    return np.nan

df['ALCDAY5'] = df['ALCDAY5'].apply(alco_days_to_weekly)


Trasformo le colonne in binarie per migliore interpretazione

In [ ]:
df['BPHIGH4'].unique()
print(df['BPHIGH4'].value_counts(dropna=False))


print(df['PERSDOC2'].value_counts(dropna=False))


In [ ]:
df['PERSDOC2'] = pd.Series(np.select(
    [
        df['PERSDOC2'].isin([1, 2]),  # rischio
        df['PERSDOC2'] == 3           # no rischio
    ],
    [
        1,  # rischio
        0   # no rischio
    ],
    default=np.nan  # incerti, rifiutati, mancanti
), index=df.index).astype('Int64')  # nullable integer per compatibilità con imputazione

df['BPHIGH4'] = pd.Series(np.select(
    [
        df['BPHIGH4'].isin([1, 2, 4]),  # rischio
        df['BPHIGH4'] == 3              # no rischio
    ],
    [
        1,  # rischio
        0   # no rischio
    ],
    default=np.nan  # incerti, rifiutati, mancanti
), index=df.index).astype('Int64')  # nullable integer per compatibilità con imputazione


In [ ]:
df.info()

Trasformo nelle colonne binarie la codifica di no (2) in 0

In [ ]:
df['BLOODCHO'].unique()

In [ ]:



def to_binary(series, yes_value=1, no_value=2):
    """
    Trasforma una Serie pandas con codifica 1=yes, 2=no in 0/1.
    - yes_value  → 1
    - no_value   → 0
    - tutti gli altri valori → np.nan
    """
    return series.map({yes_value: 1, no_value: 0}).astype('Int64')

# Esempio di utilizzo:
binary_cols = ['HLTHPLN1','MEDCOST','BLOODCHO','TOLDHI2','CVDINFR4',
               'CVDCRHD4','CVDSTRK3','ASTHMA3','CHCSCNCR','CHCOCNCR',
               'CHCCOPD1','HAVARTH3','ADDEPEV2','CHCKIDNY','VETERAN3',
               'INTERNET','QLACTLM2','USEEQUIP','BLIND','DECIDE',
               'DIFFWALK','DIFFDRES','DIFFALON','SMOKE100','EXERANY2','FLUSHOT6',
               'PNEUVAC3', 'HIVTST6'] 

for col in binary_cols:
    if col in df.columns:
        df[f'{col}'] = to_binary(df[col])
        print(df[f'{col}'].unique())
    else:
        print(f"Colonna `{col}` non trovata nel DataFrame. Operazione saltata.")
        
        
        


In [ ]:
df['BLOODCHO'].unique()

In alcune variabili la codifica dei valori non lo so/ rifiutato è differente. I valori con questo significato verranno trasformati in valori Nan. La loro frequenza è bassa.

In [ ]:
# Lista dei codici da trattare come NaN
missing_codes = [7, 9]

# Elenco delle colonne *da saltare* (quelle in cui quei codici non sono missing)
exceptions = [
    'PHYSHLTH', 'MENTHLTH', 'CHILDREN', 'NUMADULT_2',
    '_STATE', 'EMPLOY1', 'INCOME2', '_RACE'
]

# Calcolo le colonne *su cui* voglio fare il replace
cols_to_clean = [c for c in df.columns if c not in exceptions]

# Applico il replace solo su quelle
df[cols_to_clean] = df[cols_to_clean].replace(missing_codes, np.nan)


#in employ1 e _race togliere solo il 9 

#da income2, physhlth, menthlth,  togliere 77 e 99
#in children 88 = 0 e togliere 99

# 1) EMPLOY1 e _RACE: codice “9” → NaN, poi category
df[['EMPLOY1','_RACE']] = (
    df[['EMPLOY1','_RACE']]
    .replace(9, np.nan)
)

# 2) INCOME2, PHYSHLTH, MENTHLTH: codici “77” e “99” → NaN
#    - Income2 resta categorica
#    - PHYSHLTH e MENTHLTH sono numeriche (giorni), quindi float
df['INCOME2'] = (
    df['INCOME2']
    .replace([77,99], np.nan)
)
df[['PHYSHLTH','MENTHLTH']] = (
    df[['PHYSHLTH','MENTHLTH']]
    .replace({77: np.nan, 99: np.nan, 88: 0})
)

# 3) CHILDREN: codice “88” → 0 (nessun bambino), “99” → NaN
df['CHILDREN'] = (
    df['CHILDREN']
    .replace({88: 0, 99: np.nan})
)


In [ ]:
# Calcola la frazione di NaN per colonna e moltiplica per 100
nan_perc = df.isna().mean() * 100

# Stampa nome colonna e percentuale formattata
for col, perc in nan_perc.items():
    print(f"{col}: {perc:.2f}% NaN")


Cambiamo adesso il tipo delle variabili

In [ ]:
"""
from pandas.api.types import CategoricalDtype


nominal_cols = [
    '_STATE','SEX','MARITAL','EMPLOY1','_RACE', '_BMI5CAT'
]

# definiamo quali ordinali hanno codifica "1 = best, ↑ = worse"
ordinal_asc = {
    'GENHLTH':       [1,2,3,4,5],       # 1=Excellent … 5=Poor
    'CHECKUP1':      [1,2,3,4,5,6,7,8], # 
    'CHOLCHK':       [1,2,3,4],         
    '_AGE_G':        [1,2,3,4,5,6],     # 1=18–24 … 6=65+
    '_PACAT1':       [1,2,3,4],         # 1=High active … 4=Inactive
    'SEATBELT':      [1,2,3,4,5]        # 1=Always … 5=Never (invertito)
}

# e quali ordinali hanno codifica "1 = worst, ↑ = better" (e.g. lower code = peggiore)
ordinal_desc = {
    'EDUCA':         [6,5,4,3,2,1],     # 1=None … 6=Post-grad (invertito)
    'INCOME2':       [8,7,6,5,4,3,2,1],     # 1=Less than $10K … 6=$75K or more
    '_SMOKER3':      [4,3,2,1] ,         # 1=Current every day … 4=Never
    'USENOW3':       [3,2,1]          # 1=Every day … 3=Not at all (invertito)
}

# 2) Cast nominali a 'category' (manteniamo i codici numerici)
for col in nominal_cols:
    df[col] = df[col].astype('category')

# 3) Cast ordinali a 'category' con ordered=True
for col, cats in ordinal_asc.items():
    cat_type = CategoricalDtype(categories=cats, ordered=True)
    df[col] = df[col].astype(cat_type)

for col, cats in ordinal_desc.items():
    cat_type = CategoricalDtype(categories=cats, ordered=True)
    df[col] = df[col].astype(cat_type)

# 4) Le vere variabili continue in float
numeric_cols = [
    'PHYSHLTH','MENTHLTH','CHILDREN','NUMADULT_2',
    'ALCDAY5', 'FRUITJU1','FRUIT1', 'FVBEANS','FVGREEN',
    'FVORANG','VEGETAB1',  'STRENGTH'
]

df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# 5) Controllo finale
print("Dtypes finali:")
print(df[nominal_cols + list(ordinal_asc) + list(ordinal_desc) + numeric_cols].dtypes)
"""

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.to_csv('cleaned_data.csv', index=False)

In [ ]:
df['CHILDREN'].unique()
df['NUMADULT_2'].unique()
